# UEFA European Football Championship

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing libraries and defining necessary variables

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

URL_FBREF = "https://fbref.com"
UEFA_EURO_URL = "https://fbref.com/en/comps/676/history/European-Championship-Seasons"

## General information on each euro competition from 2000 to 2024

> Ajouter une citation






### By using [FBREF](https://fbref.com/en/comps/676/history/European-Championship-Seasons)

*   Year
*   Host Country
*   Number of teams in the competition
*   Winner of the competition
*   Runner-up of the competition
*   URL of each competitions logo

In [ ]:
df_euro_seasons = pd.read_html('https://fbref.com/en/comps/676/history/European-Championship-Seasons',
                               attrs={'id':'seasons'})[0].drop(['Competition Name', 'Final', 'Top Scorer'], axis=1)

In [ ]:
response_fbref = requests.get(UEFA_EURO_URL)
html_fbref = response_fbref.content
soup_fbref = bs(html_fbref, "lxml")

In [ ]:
get_euro_seasons_table = soup_fbref.find(attrs={'id':'seasons'})
get_euro_seasons_competition_url = []

def retrieveHrefElement(element, json_attrs, url_fbref):
  element = element.find(attrs=json_attrs)
  href_element = element.find('a', href=True)
  if href_element:
    return url_fbref + href_element['href']
  return 'NaN'

row = 0
for element in get_euro_seasons_table.find('tbody').find_all('tr'):
  # Retrieve the FBREF URL for each euro competition
  get_euro_seasons_competition_url.append(retrieveHrefElement(element, {'data-stat':'year'}, URL_FBREF))

In [ ]:
def retrieveElements(url, json_attrs):
  response = requests.get(url)
  html = response.content
  soup = bs(html, "lxml")
  return soup.find(attrs=json_attrs)

get_euros_logo_url = []
get_most_cleaned_sheets_player_names = []
get_number_of_clean_sheets = []

for url in get_euro_seasons_competition_url:
  # Retrieve the url for each competition's logo
  get_euros_logo_url.append(retrieveElements(url, {'class':'teamlogo'})['src'])


df_euro_seasons['Logos URL'] = get_euros_logo_url
df_euro_seasons["Logos URL"][0] = "https://upload.wikimedia.org/wikipedia/fr/0/0a/Logo_UEFA_Euro_2024.svg"

<ipython-input-140-7ab189616d85>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_euro_seasons["Logos URL"][0] = "https://upload.wikimedia.org/wikipedia/fr/0/0a/Logo_UEFA_Euro_2024.svg"


### By manually adding data

*   Total number of matches played in each tournament
*   Total revenue collected in each competition

In [ ]:
# Adding data manually

number_of_matches = [51, 51, 51, 31, 31, 31, 31]

total_revenue = [np.nan, 1882.5, 1916, 1390.9, 1350.9, 855.2, 229.9]

df_euro_seasons["Host Country"][1] = "Europe"
df_euro_seasons["Total revenue (in million euros)"] = total_revenue
df_euro_seasons["Matches"] = number_of_matches

<ipython-input-141-f834fd39fba5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_euro_seasons["Host Country"][1] = "Europe"


### By using [WIKIPEDIA](https://en.wikipedia.org/wiki/UEFA_European_Championship_records_and_statistics#Team:_Tournament_position)

*   Winning coach
*   Player of the Tournament
*   Number of clean sheets
*   Top Scorers number of goals
*   Top Scorers

In [ ]:
df_additional_stats = pd.read_html("https://en.wikipedia.org/wiki/UEFA_European_Championship_records_and_statistics#Attendance",
                                   match="Player of the Tournament")[0].drop(["Hosts", "Champions"], axis=1)
df_additional_stats.loc[15,'Year'] = 2021

In [ ]:
df_additional_stats = df_additional_stats[(df_additional_stats["Year"] >= 2000) & (df_additional_stats["Year"] <= 2024)]
df_euro_seasons = df_euro_seasons.merge(df_additional_stats, on="Year")

In [ ]:
player_names = []
player_goals = []

for top_scorer in df_additional_stats["Top scorer(s) (goals)"]:
  player_names.append([name.split('(')[0].strip() for name in top_scorer.split(')')][:-1])
  player_goals.append([goals.split('(')[-1].strip() for goals in top_scorer.split(')')][0])

player_names[-1] = "To be determined"
df_euro_seasons['Top Scorer Names'] = player_names
df_euro_seasons['Top Scorer Number of Goals'] = player_goals
df_euro_seasons.drop(['Top scorer(s) (goals)'], axis=1, inplace=True)

In [ ]:
df_euro_seasons.to_csv('/content/drive/MyDrive/EURO 24 Project/data/euro_seasons.csv', index=False)

## Specific metrics to each euro competition from 2000 to 2024


### Euro - 2000

#### Euro 2000 - [From Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2000_statistics)

- **Pld** = total games played
- **W** = total games won
- **D** = total games drawn (tied)
- **L** = total games lost
- **Pts** = total points accumulated (teams receive three points for a win, one point for a draw and no points for a loss)
- **APts** = average points per game
- **GF** = total goals scored (goals for)
- **AGF** = average goals scored per game
- **GA** = total goals conceded (goals against)
- **AGA** = average goals conceded per game
- **GD** = goal difference (GF−GA)
- **CS** = clean sheets
- **ACS** = average clean sheets
- **YC** = yellow cards
- **AYC** = average yellow cards
- **RC** = red cards
- **ARC** = average red cards

In [ ]:
df_euro_2000_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2000_statistics",
                                         match='Nation')[0].sort_values(by='Pts', ascending=False, ignore_index=True)
df_euro_2000_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2000/euro_season_2000_league_table.csv', index=False)

#### Euro 2000 - [From FBREF](https://fbref.com/en/comps/676/2000/2000-European-Championship-Stats)

- Top Team Scorer
- Group stage and knockout scores & fixtures

In [ ]:
df_euro_2000_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2000/2000-European-Championship-Stats",
                                               attrs={'id':'results20006760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2000_league_table_fbref['Squad'])]
df_euro_2000_league_table_fbref['Squad'] = squad_names
df_euro_2000_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2000_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2000/euro_season_2000_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2000 = pd.read_html("https://fbref.com/en/comps/676/2000/schedule/2000-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2000_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2000 = pd.read_html("https://fbref.com/en/comps/676/2000/schedule/2000-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2000_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2000.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2000/euro_season_2000_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2000.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2000/euro_season_2000_ko_stage.csv', index=False)

### Euro - 2004

#### Euro 2004 - From [Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2004_statistics)

- Refer to abbreviations from Euro 2000 section

In [ ]:
df_euro_2004_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2004_statistics",
                                         match='Nation')[0].sort_values(by='Pts', ascending=False, ignore_index=True)
df_euro_2004_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2004/euro_season_2004_league_table.csv', index=False)

#### Euro 2004 - From [FBREF](https://fbref.com/en/comps/676/2004/2004-European-Championship-Stats)
- Same data collected as for Euro 2000

In [ ]:
df_euro_2004_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2004/2004-European-Championship-Stats",
                                               attrs={'id':'results20046760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2004_league_table_fbref['Squad'])]
df_euro_2004_league_table_fbref['Squad'] = squad_names
df_euro_2004_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2004_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2004/euro_season_2004_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2004 = pd.read_html("https://fbref.com/en/comps/676/2004/schedule/2004-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2004_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2004 = pd.read_html("https://fbref.com/en/comps/676/2004/schedule/2004-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2004_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2004.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2004/euro_season_2004_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2004.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2004/euro_season_2004_ko_stage.csv', index=False)

### Euro - 2008

#### Euro 2008 - From [Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2008_statistics)

In [ ]:
df_euro_2008_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2008_statistics",
                                         match='Nation')[0].sort_values(by='Pts', ascending=False, ignore_index=True)
df_euro_2008_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2008/euro_season_2008_league_table.csv', index=False)

#### Euro 2008 - From [FBREF](https://fbref.com/en/comps/676/2008/2008-European-Championship-Stats)

In [ ]:
df_euro_2008_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2008/2008-European-Championship-Stats",
                                               attrs={'id':'results20086760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2008_league_table_fbref['Squad'])]
df_euro_2008_league_table_fbref['Squad'] = squad_names
df_euro_2008_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2008_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2008/euro_season_2008_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2008 = pd.read_html("https://fbref.com/en/comps/676/2008/schedule/2008-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2008_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2008 = pd.read_html("https://fbref.com/en/comps/676/2008/schedule/2008-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2008_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2008.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2008/euro_season_2008_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2008.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2008/euro_season_2008_ko_stage.csv', index=False)

### Euro - 2012

#### Euro 2012 - From [Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2012_statistics)

In [ ]:
df_euro_2012_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2012_statistics",
                                         match='Pts')[0].sort_values(by='Pts', ascending=False, ignore_index=True).rename(columns={'Team':'Nation'})
df_euro_2012_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2012/euro_season_2012_league_table.csv', index=False)

#### Euro 2012 - From [FBREF](https://fbref.com/en/comps/676/2012/2012-European-Championship-Stats)

In [ ]:
df_euro_2012_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2012/2012-European-Championship-Stats",
                                               attrs={'id':'results20126760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2012_league_table_fbref['Squad'])]
df_euro_2012_league_table_fbref['Squad'] = squad_names
df_euro_2012_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2012_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2012/euro_season_2012_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2012 = pd.read_html("https://fbref.com/en/comps/676/2012/schedule/2012-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2012_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2012 = pd.read_html("https://fbref.com/en/comps/676/2012/schedule/2012-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2012_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2012.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2012/euro_season_2012_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2012.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2012/euro_season_2012_ko_stage.csv', index=False)

### Euro - 2016

#### Euro 2016 - From [Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2016_statistics)

In [ ]:
df_euro_2016_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2016_statistics",
                                         match='Pts')[0].sort_values(by='Pts', ascending=False, ignore_index=True).rename(columns={'Team':'Nation'})
df_euro_2016_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2016/euro_season_2016_league_table.csv', index=False)

#### Euro 2016 - From [FBREF](https://fbref.com/en/comps/676/2016/2016-European-Championship-Stats)

In [ ]:
df_euro_2016_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2016/2016-European-Championship-Stats",
                                               attrs={'id':'results20166760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2016_league_table_fbref['Squad'])]
df_euro_2016_league_table_fbref['Squad'] = squad_names
df_euro_2016_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2016_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2016/euro_season_2016_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2016 = pd.read_html("https://fbref.com/en/comps/676/2016/schedule/2016-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2016_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2016 = pd.read_html("https://fbref.com/en/comps/676/2016/schedule/2016-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2016_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2016.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2016/euro_season_2016_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2016.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2016/euro_season_2016_ko_stage.csv', index=False)

### Euro - 2020

#### Euro 2020 - From [Wikipedia](https://en.wikipedia.org/wiki/UEFA_Euro_2020_statistics)

In [ ]:
df_euro_2020_league_table = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2020_statistics",
                                         match='Pts')[0].sort_values(by='Pts', ascending=False, ignore_index=True).rename(columns={'Team':'Nation'})
df_euro_2020_league_table.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2020/euro_season_2020_league_table.csv', index=False)

#### Euro 2020 - From [FBREF](https://fbref.com/en/comps/676/2021/2021-European-Championship-Stats)

In [ ]:
df_euro_2021_league_table_fbref = pd.read_html("https://fbref.com/en/comps/676/2021/2021-European-Championship-Stats",
                                               attrs={'id':'results20216760_overall'})[0].dropna(axis=0, how='all', ignore_index=True)[['Rk', 'Squad', 'Top Team Scorer']]
squad_names = [name.split()[-1].strip() for name in list(df_euro_2021_league_table_fbref['Squad'])]
df_euro_2021_league_table_fbref['Squad'] = squad_names
df_euro_2021_league_table_fbref.rename(columns={'Squad':'Nation'}, inplace=True)
df_euro_2021_league_table_fbref.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2020/euro_season_2020_team_top_scorer.csv', index=False)

In [ ]:
df_scores_fixtures_group_stage_euro_2021 = pd.read_html("https://fbref.com/en/comps/676/2021/schedule/2021-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2021_676_1'})[0].dropna(axis=0, how='all', ignore_index=True).drop(['Notes', 'Match Report'], axis=1)
df_scores_fixtures_ko_stage_euro_2021 = pd.read_html("https://fbref.com/en/comps/676/2021/schedule/2021-European-Championship-Scores-and-Fixtures",
                                            attrs={'id':'sched_2021_676_2'})[0].dropna(axis=0, how='all', ignore_index=True)
df_scores_fixtures_group_stage_euro_2021.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2020/euro_season_2020_group_stage.csv', index=False)
df_scores_fixtures_ko_stage_euro_2021.to_csv('/content/drive/MyDrive/EURO 24 Project/data/EURO 2020/euro_season_2020_ko_stage.csv', index=False)

## Specific metrics to each team competing in the euro tournaments


### General Stats
- Team
- Appearances
- Debut
- Most recent
- Best result

In [ ]:
team_general_stats = pd.read_html("https://en.wikipedia.org/wiki/UEFA_European_Championship_records_and_statistics",
                                  match='Best result')[0]
team_general_stats.to_csv('/content/drive/MyDrive/EURO 24 Project/data/Teams/team_general_stats.csv', index=False)